In [2]:
# --- Import des bibliothèques ---
import pandas as pd
import os
import h5py
import numpy as np
import pickle
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

# --- Parcourir les fichiers du dossier ---
root_dataset = '/content/drive/MyDrive/defi-ia-2025-cnes'
for dirname, _, filenames in os.walk(root_dataset):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Import des fichiers
spectra_train = np.load(f'{root_dataset}/spectra.npy')
auxiliary_train = pd.read_csv(f'{root_dataset}/auxiliary.csv')
targets_train = pd.read_csv(f'{root_dataset}/targets.csv')

print("spectra_train shape:", spectra_train.shape)
print("auxiliary_train shape:", auxiliary_train.shape)
print("targets_train shape:", targets_train.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/defi-ia-2025-cnes/spectra.npy
/content/drive/MyDrive/defi-ia-2025-cnes/spectra_test.npy
/content/drive/MyDrive/defi-ia-2025-cnes/auxiliary_test.csv
/content/drive/MyDrive/defi-ia-2025-cnes/auxiliary.csv
/content/drive/MyDrive/defi-ia-2025-cnes/targets.csv
spectra_train shape: (3000, 52, 3)
auxiliary_train shape: (3000, 5)
targets_train shape: (3000, 3)


In [3]:
spectra_data = spectra_train[:,:,1]
error_data = spectra_train[:,:,2]

In [4]:
RJUP = 69911000
MJUP = 1.898e27
RSOL = 696340000
MSOL = 1.9891e30

In [5]:
auxiliary_train["planet_surface_gravity"] = 20
# calcul du rayon théorique
auxiliary_train["th_radius"] = np.sqrt(auxiliary_train["planet_mass_kg"]*6.67e-11/auxiliary_train["planet_surface_gravity"])/RJUP
auxiliary_train['star_radius'] = auxiliary_train['star_radius_m']/RSOL # Changement de mètre en proprotion du rayon solaire

auxiliary_train['planet_mass'] = auxiliary_train['planet_mass_kg']/MJUP # Changement de kilos vers proportion de masse de jupiter
auxiliary_train['star_mass'] = auxiliary_train['star_mass_kg']/MSOL # Changement de kilos vers proportion de masse solaire


useful_aux_columns = ["star_radius", "star_temperature", "planet_mass", "semi_major_axis_m", "star_mass", "th_radius"]


auxiliary_train = auxiliary_train[useful_aux_columns]
auxiliary_train


,star_radius,star_temperature,planet_mass,semi_major_axis_m,star_mass,th_radius
0,0.710002,5033.0,0.066516,2.277510e+10,0.789722,0.293505
1,0.850002,5320.0,0.020861,1.217744e+10,0.859697,0.164369
2,0.550002,3985.0,0.047197,1.057672e+10,0.579796,0.247234
3,0.210001,2988.0,0.020074,1.431672e+10,0.189933,0.161240
4,0.640002,4925.0,0.044051,1.795200e+10,0.689757,0.238851
...,...,...,...,...,...,...
2995,0.850002,5320.0,0.020861,1.217744e+10,0.859697,0.164369
2996,0.650002,4316.0,0.012145,1.171368e+10,0.649771,0.125417
2997,1.120003,6169.0,0.083602,4.425168e+10,1.099613,0.329047
2998,0.550002,3985.0,0.047197,1.057672e+10,0.579796,0.247234


In [6]:
import numpy as np
from sklearn.utils import shuffle

from sklearn.utils import shuffle
import numpy as np

# Augmentation
def physical_augmentation_internal(signal, error, aux, targets, multiplier=1, noise_factor=0.1):
    """
    Augmente les données et renvoie 4 tableaux : Signal, Erreur, Aux, Targets.
    Indispensable pour que le code suivant fonctionne.
    """
    n_samples, n_features = signal.shape
    total_size = n_samples * (multiplier + 1)

    new_signals = np.empty((total_size, n_features), dtype=np.float32)
    new_errors  = np.empty((total_size, n_features), dtype=np.float32) # Le 4ème élément manquant !
    new_aux     = np.empty((total_size, aux.shape[1]), dtype=np.float32)

    if targets.ndim == 1: new_targets = np.empty((total_size,), dtype=targets.dtype)
    else: new_targets = np.empty((total_size, targets.shape[1]), dtype=targets.dtype)

    # Originaux
    new_signals[:n_samples] = signal
    new_errors[:n_samples]  = error
    new_aux[:n_samples]     = aux
    new_targets[:n_samples] = targets

    # Copies bruitées
    for i in range(multiplier):
        start, end = n_samples * (i + 1), n_samples * (i + 2)

        # Bruit basé sur l'erreur instrumentale
        noise = np.random.randn(n_samples, n_features) * error * noise_factor

        aug_signal = np.maximum(signal + noise, 0)

        new_signals[start:end] = aug_signal
        new_errors[start:end]  = error
        new_aux[start:end]     = aux
        new_targets[start:end] = targets

    return shuffle(new_signals, new_errors, new_aux, new_targets, random_state=42)

In [7]:
from sklearn.preprocessing import Normalizer, StandardScaler

def preprocess_inside_loop(X_spec, X_aux):
    # Normalisation L2 sur les spectres (indispensable pour l'intensité)
    norm = Normalizer(norm='l2')
    X_spec_norm = norm.transform(X_spec)

    # Standardisation sur les auxiliaires
    scaler = StandardScaler()
    X_aux_norm = scaler.fit_transform(X_aux) # Fit sur le batch courant

    # Concaténation
    return np.hstack([X_spec_norm, X_aux_norm]), scaler # On renvoie le scaler si besoin pour le test

In [8]:
X_spectres_np = np.asarray(spectra_data)
X_error_np    = np.asarray(error_data)
X_aux_np      = np.asarray(auxiliary_train)
Y_train_np    = np.asarray(targets_train.values[:,1:])

**CNN-1D**

In [9]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [10]:
def get_minimal_stats(spectra, errors):
    """
    6 Statistiques Clés uniquement.
    Pas de physique, pas de dérivées.
    """
    # Moyenne & Ecart-type (L'intensité globale)
    f_mean = np.mean(spectra, axis=1, keepdims=True)
    f_std  = np.std(spectra, axis=1, keepdims=True)

    # Max & Min (L'amplitude brute)
    f_max  = np.max(spectra, axis=1, keepdims=True)
    f_min  = np.min(spectra, axis=1, keepdims=True)

    # Pente (Indicateur de diffusion Rayleigh vs Nuage plat)
    f_slope = (spectra[:, -1] - spectra[:, 0]).reshape(-1, 1)

    ratio_depth = f_min / (f_max + 1e-9)            # Profondeur
    ratio_amp   = (f_max - f_min) / (f_mean + 1e-9) # Contraste relatif

    # SNR Moyen (Indicateur de fiabilité)
    e_mean = np.mean(errors, axis=1, keepdims=True)

    return np.hstack([f_mean, f_std, f_max, f_min, f_slope, e_mean])

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, regularizers

# C'est le "Turbo". Il permet au réseau de focaliser sur les pics précis.
def cbam_block(input_tensor, ratio=8):
    filters = input_tensor.shape[-1]

    avg_pool = layers.GlobalAveragePooling1D()(input_tensor)
    max_pool = layers.GlobalMaxPooling1D()(input_tensor)

    shared_dense1 = layers.Dense(filters // ratio, activation='relu', use_bias=False)
    shared_dense2 = layers.Dense(filters, use_bias=False)

    avg_out = shared_dense2(shared_dense1(avg_pool))
    max_out = shared_dense2(shared_dense1(max_pool))

    channel_attention = layers.Activation('sigmoid')(layers.Add()([avg_out, max_out]))
    channel_attention = layers.Reshape((1, filters))(channel_attention)

    x = layers.Multiply()([input_tensor, channel_attention])

    # SPATIAL ATTENTION

    avg_pool_s = layers.Lambda(lambda t: tf.reduce_mean(t, axis=-1, keepdims=True))(x)
    max_pool_s = layers.Lambda(lambda t: tf.reduce_max(t, axis=-1, keepdims=True))(x)

    concat = layers.Concatenate(axis=-1)([avg_pool_s, max_pool_s])

    # Convolution large (7) pour repérer les motifs spectraux
    spatial_attention = layers.Conv1D(1, 7, padding='same', activation='sigmoid', use_bias=False)(concat)

    return layers.Multiply()([x, spatial_attention])

def residual_block_cbam(x, filters, kernel_size=3, stride=1):
    shortcut = x

    # Conv 1
    x = layers.Conv1D(filters, kernel_size, strides=stride, padding='same',
                      kernel_regularizer=regularizers.l2(1e-5))(x) # L2 très faible
    x = layers.BatchNormalization()(x)
    x = layers.Activation('swish')(x)

    # Conv 2
    x = layers.Conv1D(filters, kernel_size, strides=1, padding='same',
                      kernel_regularizer=regularizers.l2(1e-5))(x)
    x = layers.BatchNormalization()(x)

    # Injection de l'attention
    x = cbam_block(x)

    # Adaptation Shortcut
    if stride > 1 or shortcut.shape[-1] != filters:
        shortcut = layers.Conv1D(filters, 1, strides=stride, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.Activation('swish')(x)
    return x

def create_high_performance_model(input_shape_spec, input_shape_phys):

    input_spec = layers.Input(shape=input_shape_spec, name="input_spec")

    # Bruit très lége, pour la stabilité numérique
    x = layers.GaussianNoise(0.02)(input_spec)

    x = layers.Conv1D(64, 5, padding='same', activation='swish')(x)
    x = layers.BatchNormalization()(x)

    # ResNet Backbone

    x = residual_block_cbam(x, 64, stride=1)
    x = layers.SpatialDropout1D(0.15)(x) # 15% des filtres éteints

    x = residual_block_cbam(x, 128, stride=2)
    x = layers.SpatialDropout1D(0.3)(x)

    x = residual_block_cbam(x, 256, stride=2)
    x = layers.SpatialDropout1D(0.5)(x)

    x = residual_block_cbam(x, 256, stride=1)
    x = layers.SpatialDropout1D(0.5)(x)

    avg_pool = layers.GlobalAveragePooling1D()(x)
    max_pool = layers.GlobalMaxPooling1D()(x)
    y = layers.Concatenate()([avg_pool, max_pool])

    # Ajout de la physique
    input_phys = layers.Input(shape=input_shape_phys, name="input_phys")
    z = layers.Dense(64, activation='swish')(input_phys)
    z = layers.BatchNormalization()(z)

    concat = layers.Concatenate()([y, z])

    # Dense Layers
    x = layers.Dense(512, activation='swish', kernel_regularizer=regularizers.l2(1e-5))(concat)
    x = layers.Dropout(0.3)(x) # Juste assez pour éviter le par coeur

    x = layers.Dense(128, activation='swish', kernel_regularizer=regularizers.l2(1e-5))(x)
    x = layers.Dropout(0.2)(x)

    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs=[input_spec, input_phys], outputs=outputs)

    loss_fn = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.01)

    optimizer = optimizers.AdamW(
    learning_rate=1e-3,
    weight_decay=1e-4
)
    model.compile(optimizer=optimizer,
                  loss=loss_fn,
                  metrics=['accuracy'])
    return model

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import callbacks
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)

early_stop = EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True)

N_SPLITS = 5

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
# Attention bloc

def train_deep_resnet_cv(raw_spec, raw_err, raw_aux, y_target):

    # Initialisation
    N_SPLITS = 3
    skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

    oof_preds = np.zeros(len(y_target))
    models_list = [] # Pour stocker (modèle, scaler) et faire du TTA plus tard

    # Gestion format auxiliaire
    if hasattr(raw_aux, 'values'): aux_values = raw_aux.values
    else: aux_values = raw_aux

    print(f"🚀 Training Deep ResNet (High Perf) sur {len(y_target)} samples...")

    for fold, (train_idx, val_idx) in enumerate(skf.split(raw_spec, y_target)):

        # Préparation des données
        X_s_t, X_e_t, X_a_t = raw_spec[train_idx], raw_err[train_idx], aux_values[train_idx]
        Y_t = y_target[train_idx]

        X_s_v, X_e_v, X_a_v = raw_spec[val_idx], raw_err[val_idx], aux_values[val_idx]
        Y_v = y_target[val_idx]

        # Augmentation
        X_s_t_aug, X_e_t_aug, X_a_t_aug, Y_t_aug = physical_augmentation_internal(
            X_s_t, X_e_t, X_a_t, Y_t, multiplier=6, noise_factor=0.2
        )

        # Preprocessing
        stats_t = get_minimal_stats(X_s_t_aug, X_e_t_aug)
        X_scalar_train = np.hstack([X_a_t_aug, stats_t])

        stats_v = get_minimal_stats(X_s_v, X_e_v)
        X_scalar_val = np.hstack([X_a_v, stats_v])

        scaler = StandardScaler()
        X_scalar_train_sc = scaler.fit_transform(X_scalar_train)
        X_scalar_val_sc   = scaler.transform(X_scalar_val)

        # Normalisation
        mean_t = np.mean(X_s_t_aug, axis=1, keepdims=True)
        std_t  = np.std(X_s_t_aug, axis=1, keepdims=True)
        X_spec_train_norm = ((X_s_t_aug - mean_t) / (std_t + 1e-6)).reshape(-1, 52, 1)

        mean_v = np.mean(X_s_v, axis=1, keepdims=True)
        std_v  = np.std(X_s_v, axis=1, keepdims=True)
        X_spec_val_norm = ((X_s_v - mean_v) / (std_v + 1e-6)).reshape(-1, 52, 1)

        #  Modèle et callbackcs
        model = create_high_performance_model((52, 1), (X_scalar_train_sc.shape[1],))

        checkpoint_path = f"best_h2o_fold{fold}.keras"
        chkpt = ModelCheckpoint(
            checkpoint_path,
            save_best_only=True,
            monitor='val_accuracy',  # On vise le score, pas la loss !
            mode='max',
            verbose=0
        )

        # Scheduler : on réduit le lr si l'accuracy stagne
        reduce_lr = ReduceLROnPlateau(
            monitor='val_accuracy',
            mode='max',
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1
        )

        # Early stopping
        early_stop = EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True
        )

        print(f"🏎️ Début Course Performance Fold {fold+1}...")

        # Entrainement
        history = model.fit(
            [X_spec_train_norm, X_scalar_train_sc], Y_t_aug,
            validation_data=([X_spec_val_norm, X_scalar_val_sc], Y_v),
            epochs=60,
            batch_size=64,
            callbacks=[chkpt, reduce_lr, early_stop],
            verbose=1
        )

        # On garde le meilleur modèle
        model.load_weights(checkpoint_path)

        # Prédiction et sauvegarde
        val_preds = model.predict([X_spec_val_norm, X_scalar_val_sc], verbose=0).flatten()
        oof_preds[val_idx] = val_preds

        models_list.append({
            'model': model,
            'scaler': scaler,
            'fold': fold
        })

        acc = np.mean((val_preds >= 0.5) == Y_v)

    total_acc = np.mean((oof_preds >= 0.5) == y_target)
    print(f"Moyenne Accuracy Globale : {total_acc:.4f}")

    return oof_preds, models_list

In [16]:
root_dataset = '/content/drive/MyDrive/defi-ia-2025-cnes'

spectra_test = np.load(f'{root_dataset}/spectra_test.npy')
test_spectra = spectra_test[:,:,1]
test_error = spectra_test[:,:,2]
auxiliary_test = pd.read_csv(f'{root_dataset}/auxiliary_test.csv')

auxiliary_test["planet_surface_gravity"] = 20
# calcul du rayon théorique
auxiliary_test["th_radius"] = np.sqrt(auxiliary_test["planet_mass_kg"]*6.67e-11/auxiliary_test["planet_surface_gravity"])/RJUP
auxiliary_test['star_radius'] = auxiliary_test['star_radius_m']/RSOL # Changement de mètre en proprotion du rayon solaire

auxiliary_test['planet_mass'] = auxiliary_test['planet_mass_kg']/MJUP # Changement de kilos vers proportion de masse de jupiter
auxiliary_test['star_mass'] = auxiliary_test['star_mass_kg']/MSOL # Changement de kilos vers proportion de masse solaire


useful_aux_columns = ["star_radius", "star_temperature", "planet_mass", "semi_major_axis_m", "star_mass", "th_radius"]


test_aux = auxiliary_test[useful_aux_columns].values
auxiliary_test[useful_aux_columns]

,star_radius,star_temperature,planet_mass,semi_major_axis_m,star_mass,th_radius
0,0.510001,3476.0,0.019760,4.293520e+09,0.509821,0.159971
1,0.650002,4316.0,0.012145,1.171368e+10,0.649771,0.125417
2,0.640002,4925.0,0.044051,1.795200e+10,0.689757,0.238851
3,0.380001,3506.0,0.015040,1.078616e+10,0.389863,0.139565
4,0.210001,2988.0,0.020074,1.431672e+10,0.189933,0.161240
...,...,...,...,...,...,...
1027,0.650002,4316.0,0.012145,1.171368e+10,0.649771,0.125417
1028,0.600002,4305.0,0.004090,5.871800e+09,0.609785,0.072784
1029,0.380001,3506.0,0.015040,1.078616e+10,0.389863,0.139565
1030,0.650002,4316.0,0.012145,1.171368e+10,0.649771,0.125417


In [17]:
from tensorflow.keras import callbacks

# Préparation
X_flux = spectra_train[:, :, 1]
X_err  = spectra_train[:, :, 2]
X_flux_test = spectra_test[:, :, 1]
X_err_test  = spectra_test[:, :, 2]

#H20
print("\n=== 🌊 DEEP RESNET H2O ===")
oof_h2o_deep, arts_h2o_deep = train_deep_resnet_cv(
    X_flux, X_err, auxiliary_train, targets_train['eau']
)

#Clouds
oof_clouds_deep, arts_clouds_deep = train_deep_resnet_cv(
    X_flux, X_err, auxiliary_train, targets_train['nuage']
)



=== 🌊 DEEP RESNET H2O ===
🚀 Training Deep ResNet (High Perf) sur 3000 samples...
🏎️ Début Course Performance Fold 1...
Epoch 1/60
219/219 ━━━━━━━━━━━━━━━━━━━━ 52s 97ms/step - accuracy: 0.6389 - loss: 0.7329 - val_accuracy: 0.7560 - val_loss: 0.5483 - learning_rate: 0.0010
Epoch 2/60
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9402 - loss: 0.1829 - val_accuracy: 0.9410 - val_loss: 0.1797 - learning_rate: 0.0010
Epoch 3/60
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9565 - loss: 0.1490 - val_accuracy: 0.9620 - val_loss: 0.1366 - learning_rate: 0.0010
Epoch 4/60
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9660 - loss: 0.1264 - val_accuracy: 0.9590 - val_loss: 0.1554 - learning_rate: 0.0010
Epoch 5/60
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9698 - loss: 0.1225 - val_accuracy: 0.9640 - val_loss: 0.1459 - learning_rate: 0.0010
Epoch 6/60
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9739 - loss: 0.1081 - val_accuracy: 0.9620 - val_l

NameError: name 'predict_dual_cnn_minimal' is not defined

In [26]:
# Sauvegarde pour blending
np.save("oof_h2o_deep.npy", oof_h2o_deep)
np.save("ooof_clouds_deep.npy", oof_clouds_deep)

In [21]:
from sklearn.metrics import accuracy_score

def find_best_threshold(y_true, y_pred_oof):
    best_acc = 0
    best_thresh = 0.5

    # On teste tous les seuils de 0.20 à 0.80
    for thresh in np.arange(0.2, 0.8, 0.005):
        acc = accuracy_score(y_true, (y_pred_oof >= thresh).astype(int))
        if acc > best_acc:
            best_acc = acc
            best_thresh = thresh

    return best_thresh, best_acc

thresh_h2o, acc_h2o = find_best_threshold(targets_train['eau'], oof_h2o_deep)
thresh_clouds, acc_clouds = find_best_threshold(targets_train['nuage'], oof_clouds_deep)

🎯 Seuil Optimal H2O : 0.435 (Acc Train: 0.9740)
🎯 Seuil Optimal Clouds : 0.455 (Acc Train: 0.9673)


In [18]:
import numpy as np

def predict_deep_resnet(models_list, X_flux_test, X_err_test, X_aux_test):
    """
    Predit les probabilités en utilisant l'ensemble des modèles entraînés (Folds).
    """
    # --- 1. PREPARATION AUXILIAIRE ---
    if hasattr(X_aux_test, 'values'):
        aux_values = X_aux_test.values
    else:
        aux_values = X_aux_test

    # --- 2. PREPROCESSING COMMUN (Gain de temps) ---

    # Calcul des Stats (Comme dans le Train C.1)
    stats_test = get_minimal_stats(X_flux_test, X_err_test)

    # Construction du vecteur Scalaire Brut (Aux + Stats)
    X_scalar_raw = np.hstack([aux_values, stats_test])

    # Normalisation comme au training
    mean_test = np.mean(X_flux_test, axis=1, keepdims=True)
    std_test  = np.std(X_flux_test, axis=1, keepdims=True)
    X_spec_norm = ((X_flux_test - mean_test) / (std_test + 1e-6)).reshape(-1, 52, 1)

    all_preds = []

    for i, artifact in enumerate(models_list):
        model = artifact['model']
        scaler = artifact['scaler'] # Le scaler spécifique à ce fold !

        # Scaling spécifique
        X_scalar_sc = scaler.transform(X_scalar_raw)

        # Prédiction
        pred_fold = model.predict([X_spec_norm, X_scalar_sc], verbose=0).flatten()
        all_preds.append(pred_fold)

    final_predictions = np.mean(all_preds, axis=0)

    return final_predictions

In [27]:
pred_h2o_deep = predict_deep_resnet(arts_h2o_deep, X_flux_test, X_err_test, test_aux)
pred_clouds_deep = predict_deep_resnet(arts_clouds_deep, X_flux_test, X_err_test, test_aux)



=== 🔮 PRÉDICTION TEST (DEEP) ===
🔮 Démarrage Prédiction Ensemble sur 3 modèles...
🔮 Démarrage Prédiction Ensemble sur 3 modèles...


In [22]:
# Soumission Test 100% Deep
submission_df = pd.DataFrame({
    'id': range(len(pred_h2o_deep)),
    'eau': (pred_h2o_deep >= thresh_h2o).astype(int),
    'nuage': (pred_clouds_deep >= thresh_clouds).astype(int)
})
submission_df.set_index('id', inplace=True)
submission_df.to_csv("submission_deep_resnet.csv")
print("✅ Fichier généré : submission_deep_resnet.csv")

✅ Fichier généré : submission_deep_resnet.csv


**Sauvegarde**

In [30]:
np.save(os.path.join(root_dataset, "oof_h2o_deep.npy"), oof_h2o_deep)
np.save(os.path.join(root_dataset, "oof_clouds_deep.npy"), oof_clouds_deep)

In [31]:
np.save(os.path.join(root_dataset, "pred_h2o_deep.npy"), pred_h2o_deep)
np.save(os.path.join(root_dataset, "pred_clouds_deep.npy"), pred_clouds_deep)